# Filter snowlines manually by plotting and assessing visually

In [ ]:
##### MODIFY HERE #####
# path to snow-cover-mapping
base_path = '/Users/raineyaberle/Research/PhD/snow_cover_mapping/snow-cover-mapping/'
# specify site name
site_name = 'Wolverine'
# path to study site
study_site_path = '/Users/raineyaberle/Google Drive/My Drive/Research/PhD/snow_cover_mapping/study-sites/' + site_name + '/'
# path where figure files are located
figures_out_path = study_site_path + '/figures/' 
# path to snowline files
sl_est_path = study_site_path +'imagery/snowlines/' 
# path where filtered snowlines will be saved
out_path = sl_est_path 
#######################

# -----Determine imagery folders
L_im_path = study_site_path + 'imagery/Landsat/'
PS_im_path = study_site_path + 'imagery/PlanetScope/mosaics/'
S2_SR_im_path = study_site_path + 'imagery/Sentinel-2_SR/'
S2_TOA_im_path = study_site_path + 'imagery/Sentinel-2_TOA/'

# -----Import packages
import matplotlib.pyplot as plt
import glob
import pandas as pd
import numpy as np
import pickle
import sys
import xarray as xr
import rioxarray as rxr

# -----Add path to functions
sys.path.insert(1, base_path+'functions/')
import pipeline_utils as f

# -----Load dataset dictionary
with open(base_path + 'inputs-outputs/datasets_characteristics.pkl', 'rb') as fn:
    dataset_dict = pickle.load(fn)

## Load and compile snowline CSV files

In [ ]:
# -----Compile snowline files
sl_est_fns = glob.glob(sl_est_path + '*snowline.csv')
# compile all snowline files into one DataFrame
sl_est_full = pd.DataFrame()
for fn in sl_est_fns:
    # read file
    if 'csv' in fn:
        sl_est = pd.read_csv(fn)
    elif 'pkl' in fn:
        sl_est = pickle.load(open(fn, 'rb'))
    # concatenate to df
    sl_est_full = pd.concat([sl_est_full, sl_est])
sl_est_full = sl_est_full.sort_values(by=['datetime']).reset_index(drop=True) # renumber, sort by date
if 'Unnamed: 0' in list(sl_est_full.columns):
    sl_est_full = sl_est_full.drop('Unnamed: 0', axis=1) # remove Unnamed column
sl_est_full

## Display all snowline coordinates on original input image, determine whether to remove snowline from time series

In [ ]:
checkboxes = f.manual_snowline_filter_plot(sl_est_full, dataset_dict, L_im_path, PS_im_path, S2_SR_im_path, S2_TOA_im_path)

## Grab user input, plot and save filtered snowlines CSV to file

In [ ]:
# -----Grab user inputs from checkboxes
values = np.array([x.value for x in checkboxes])
num_filt =len(np.argwhere(values==True))

# -----Filter snowlines based on user input
print(str(num_filt) + ' images selected to remove from time series.')
proceed = input('Proceed? (Y/N)')
if proceed=='Y':
    # reformat datetime column for plotting
    sl_est_full['datetime'] = sl_est_full['datetime'].astype('datetime64[ns]')
    # remove rows where values == True
    sl_est_full_filt = sl_est_full.iloc[values==False,:]
    # save removed rows for plotting
    sl_est_full_removed = sl_est_full.iloc[values==True,:]
    # plot
    fig = plt.figure(figsize=(10,6))
    plt.plot(sl_est_full_removed['datetime'], sl_est_full_removed['snowlines_elevs_median_m'], 
             'x', markersize=4, color='#969696', label='Removed points')
    plt.plot(sl_est_full_filt['datetime'], sl_est_full_filt['snowlines_elevs_median_m'], 
             '.k', markersize=8, label='Filtered timeseries')
    plt.title(site_name)
    plt.xlabel('Date')
    plt.ylabel('Median snowline elevation [m]')
    plt.legend(loc='best')
    plt.grid()
    plt.show()
    
    # -----Save figure
    min_date = str(np.nanmin(sl_est_full['datetime']))[0:10].replace('-','')
    max_date = str(np.nanmax(sl_est_full['datetime']))[0:10].replace('-','')
    fig_fn = figures_out_path + min_date + '_'+ max_date + '_' + site_name + '_filtered_snowlines_manual.png'
    fig.savefig(fig_fn, dpi=300, facecolor='w')
    print('figure saved to file: ' + fig_fn)

    # -----Save filtered snowline time series
    sl_fn = min_date + '_' + max_date + '_' + site_name + '_filtered_snowlines_manual.csv'
    sl_est_full_filt.to_csv(out_path + sl_fn)
    print('filtered snowlines saved to file: ' + out_path + sl_fn)
    
else:
    print('Not proceeding...')